In [ ]:
from datetime import datetime, date
from pyspark.sql import SparkSession, Row, functions as F
from pyspark.sql.window import Window

from typing import Iterator

In [3]:
# Spark session & context

# create it in local mode
spark = SparkSession.builder.master("local").getOrCreate()
sc = spark.sparkContext
spark.conf.set('spark.sql.repl.eagerEval.enabled', False)

In [22]:
size = 100
customer_df = spark.read.csv(f"./work/data/customers-{size}.csv", header=True, inferSchema=True)
order_df = spark.read.csv(f"./work/data/orders-from-customers-{size}.csv", header=True, inferSchema=True)

In [37]:
# find most popular month during each year for purchases
(
    order_df
        .withColumn("date", F.to_date("Purchase Date"))
        .withColumn("month", F.month("date"))
        .withColumn("year", F.year("date"))
        .drop("date")
        .groupBy("year", "month")
        .count()
        # Partition by year and order by count within partition
        .withColumn(
            "rank",
            F.rank().over(Window.partitionBy("year").orderBy(F.desc("count")))
        )
        # Only show rank 1 from each partition
        .filter(F.col("rank") == 1)
        .orderBy("year")
        .drop("rank")
        .show()
)

+----+-----+-----+
|year|month|count|
+----+-----+-----+
|2023|    4|   36|
|2024|   11|   29|
|2025|    2|   25|
+----+-----+-----+



In [42]:
# find most popular month across entire data
(
    order_df
        .withColumn("date", F.to_date("Purchase Date"))
        .withColumn(
            "month",
            F.concat_ws("-", F.lpad(F.month("date"), 2, '0'), F.year("date"))
        )
        .drop("date")
        .groupBy("month")
        .count()
        .orderBy("count", ascending=False)
        .show(3)
)

+-------+-----+
|  month|count|
+-------+-----+
|04-2023|   36|
|07-2023|   29|
|11-2024|   29|
+-------+-----+
only showing top 3 rows



In [56]:
# find the top 5 customers (name, id, number of purchases)
(
    order_df
        .join(customer_df, on='Customer Id')
        .withColumn(
            'Name',
            F.concat_ws(' ', F.col('First Name'), F.col('Last Name'))
        )
        .groupBy('Customer Id', 'Name')
        .count()
        .withColumnRenamed('count','Purchase Count')
        .select('Name', 'Customer Id', 'Purchase Count')
        .orderBy(F.desc('Purchase Count'))
        .show(5)
)

+-------------+------------+--------------+
|         Name| Customer Id|Purchase Count|
+-------------+------------+--------------+
|Jose Stephens|09238C7A079D|            21|
| Leslie Jones|DCC426900E4B|            21|
|  Trevor Kent|4BF474004F29|            20|
|Michael Ewing|B1773C62C2B1|            19|
|  John Fisher|F6AED63445DD|            19|
+-------------+------------+--------------+
only showing top 5 rows



In [ ]:
# find the top 10 customers (name, id, total amount spent, and number of purchases)
(
    order_df
        .join(customer_df, on='Customer Id')
        .withColumn(
            'Name',
            F.concat_ws(' ', F.col('First Name'), F.col('Last Name'))
        )
        .groupBy('Customer Id', 'Name')
        .agg(
            F.count('Order Id').alias('Purchase Count'),
            F.round(F.sum('Amount'), 2).alias('Total Amount'),
        )
        .select('Name', 'Customer Id', 'Purchase Count', 'Total Amount')
        .orderBy(F.desc('Purchase Count'))
        .show(10)
)

+----------------+------------+--------------+------------+
|            Name| Customer Id|Purchase Count|Total Amount|
+----------------+------------+--------------+------------+
|   Jose Stephens|09238C7A079D|            21|     1296.31|
|    Leslie Jones|DCC426900E4B|            21|     1043.66|
|     Trevor Kent|4BF474004F29|            20|     1013.57|
|     John Fisher|F6AED63445DD|            19|     1072.17|
|   Michael Ewing|B1773C62C2B1|            19|     1254.72|
|    Diane Vaughn|EB9C157925CA|            18|      987.19|
|      Diane Cole|6A2F4D7F36E5|            18|     1014.62|
|    Anita Guzman|7445FA4A5CC8|            18|     1057.67|
|   Johnny Potter|1B22A473C51A|            18|     1023.41|
|Kimberly Johnson|0C80E95EE32B|            17|      894.13|
+----------------+------------+--------------+------------+
only showing top 10 rows



In [ ]:
# find average purchases per month for customer
(
    customer_df
        .join(order_df, "Customer Id")
        .withColumn(
            'Name',
            F.concat_ws(" ", "First Name", "Last Name")
        )
        .groupBy("Customer Id", "Name")
        .agg(
            # TODO: Make this average per month:
            F.round(F.avg("Amount"), 2).alias("Average Amount")
        )
        .select("Customer Id", "Name", "Average Amount")
        .show(5)
)

+------------+-------------------+--------------+
| Customer Id|               Name|Average Amount|
+------------+-------------------+--------------+
|0C80E95EE32B|   Kimberly Johnson|          52.6|
|F6F2E7F309F0|          Anne Lane|         61.34|
|F49EEA01BB9B|  Courtney Williams|         92.04|
|4EE5CBD332E8|Brandon Fitzpatrick|         60.49|
|5E36498E2C86|     Mariah Woodard|         31.82|
+------------+-------------------+--------------+
only showing top 5 rows



In [ ]:
# find age range of customers

In [ ]:
# do same queries with SQL syntax